# Rocket flight


In [37]:
#Import
from matplotlib import pyplot
import numpy
import math
%matplotlib inline

## 1.  Introduction 

Prenons une fusée de masse $m_s$ avec une masse initiale de carburant $m_p(t=0)=m_{p0}$.

Soit $h$ la hauteur de la fusée par rapport au sol et $v$ sa vitesse.  Alors :
    \begin{equation}
        \frac{\mathrm{d}h}{\mathrm{d}t}=v.
    \end{equation}
    
De plus, si nous définissons $v_e$ comme étant la vitesse d'échappement du carburant, nous pouvons exprimer la force résultante qui s'exerce sur la fusée:$$(m_s+m_p)\frac{\mathrm{d}v}{\mathrm{d}t}=-(m_s+m_p)g+\dot{m_p} v_e - \frac{1}{2} \rho v |v| A C_D$$
    
où $\dot{m_p}=\frac{\mathrm{d}m_p}{\mathrm{d}t}$, $A$ est l'aire maximale d'une coupe horizontale de la fusée, $\rho$ la masse volumique de l'air et $C_D$ le coefficient de trainée.

Remarquons que la masse du carburant dépend du temps de la manière suivante:$$m_p = m_{p0}-\int_0^t \dot{m_p} d\tau$$ où 
\begin{equation}
    \dot{m_p}=\left\{ 
        \begin{array}{ll}
            20 ~ kg/s & \mbox{si } t \le 5 \\
            0 & \mbox{sinon.}
        \end{array}
    \right.
\end{equation}
    
Nous allons déterminer la hauteur et la vitesse de la fusée toutes les $0,1~s$.  Pour celà nous allons utiliser la methode d'Euler qui nous propose l'itération suivante:
\begin{equation}
    u_{n+1}=u_n+\Delta t . f(u_n)
\end{equation}
    
où la fonction $f$ décrit la variation de $u_i$ en fonction de $u_i$ dans le temps.  Dans notre cas, nous appliquons cette méthode à la hauteur et à la vitesse (en même temps puisqu'ils sont reliés):\begin{equation}
    \left\{
        \begin{array}{ll}
            h_{n+1}=h_n + \mathrm{d}t~ \frac{\mathrm{d}h}{\mathrm{d}t}\\
            v_{n+1}=v_n+ \mathrm{d}t~ \frac{\mathrm{d}v}{\mathrm{d}t}
        \end{array}
    \right. \\
    \Leftrightarrow ~ \left\{
        \begin{array}{ll}
            h_{n+1}=h_n + \mathrm{d}t~ v\\
            v_{n+1}=v_n+ \mathrm{d}t~ \big(-g+(\dot{m_p}v_e - \frac{1}{2} \rho v |v| A C_D)(m_s+m_p)^{-1}\big)
        \end{array}
    \right.
\end{equation}

Dans le code, ces deux données seront regroupéed dans un vecteur $u_n$.

## 2. Code

### Données :

In [38]:
m_s = 50.0 # Masse de la fusée en kg
g = 9.81 # Accélération de la pesanteur en m/s
rho = 1.091 # Densité de l'air en kg/m³
r = 0.5  # Rayon maximal d'une coupe horizontale de la fusée en m
A = (numpy.pi)*(r**2) # Surface maximale d'une section de la fusée avec un rayon de 0.5 m
v_e = 325.0 # Vitesse d'échappement du carburant en m/s
C_D = 0.15 # Coefficient de trainée
m_p0 = 100.0 # Masse du carburant au temps t = 0
h0 = 0.0 # Hauteur de la fusée au temps t = 0 en m
v0 = 0.0 # Vitesse de la fusée au temps t = 0 en m/s
m_p_point_init = 20.0 # Valeur absolue de la dérivée dans le temps de la masse du carburant au temps t = 0 en kg/s
end = 4.9 # Dernier instant pour lequel il y a du carburant à dt près en s (Remarque: Si end = 5, à l'itération qui nous donnera v_point à l'instant end+dt (=5,1s) nous ajouterions une vitesse due à la consomation de carburant alors qu'il n'y en à plus)

Définissons les fonctions (dépendantes du temps) utilisées pour décrire $\frac{\mathrm{d}v}{\mathrm{d}t}$, à savoir $\dot{m_p}$ et $m_p$.

In [39]:
def m_p_point(t) :
    """
    m_p_point correspond à la dérivée de m_p par rapport au temps
    Quand t < 5, m_p_point vaut 20.
    Quand t > 5, m_p_point vaut 0.
    """
    if t < end :
        m_p_point = m_p_point_init          
    else :
        m_p_point = 0
    return m_p_point

def integrale(t) :
    """
    Quand t < 5, l'intégrale de 0 à t vaut 20*t.
    Quand t > 5, l'intégrale de 0 à t vaut l'intégrale de 0 à 5 car l'intégrale de 5 à t est nul puisque m_p_point vaut 0.
    """
    if t < end :
        integrale = m_p_point(t)*t
    else :
        integrale = m_p_point_init*5
    return integrale

def m_p(t) :
    return m_p0 - integrale(t)


*Réponse 1:*

In [40]:
print ("Après 3.2 secondes, la fusée ne possède plus que", m_p(3.2), "kg de carburant")

Après 3.2 secondes, la fusée ne possède plus que 36.0 kg de carburant


Préparons-nous à faire nos itérations.

In [41]:
T = 40.0 # Temps maximal pour lequel on va chercher h et v en s
dt = 0.1 # Intervalle de temps entre chaque valeur cherchée en s
N = int(T/dt)+1  # Nombre de valeurs que nous allons chercher pour u = (h,v)
t = numpy.linspace(0.0, T, N)  # Toutes les valeurs t[n] que prend t selon u_n = (h_n,v_n)
u = numpy.empty((N, 2))  # On crée un vecteur u de N*2 éléments que l'on va remplir de u_n = (h_n,v_n)
u[0] = numpy.array([h0, v0])  
t0 = 0  # Temps initial en s

Définissons $f(u,t)=\frac{\mathrm{d}u}{\mathrm{d}t}$ où $u=(h,v)$ grâce à
\begin{equation}
    \frac{\mathrm{d}v}{\mathrm{d}t}=\big(-g+\frac{\dot{m_p}v_e - \frac{1}{2} \rho v |v| A C_D}{m_s+m_p}\big)
\end{equation}
et
\begin{equation}
    \frac{\mathrm{d}h}{\mathrm{d}t}=v.
\end{equation}

In [42]:
def f(u,t) :
    """
    La fonction qui décrit la variation de u_i à chaque instant t[i]
    """
    h = u[0]
    v = u[1]
    v_point = -g + ((m_p_point(t)*v_e)/(m_s + m_p(t))) - ((rho*v*abs(v)*A*C_D)/(2.0*(m_s + m_p(t))))
    return numpy.array([v, v_point])

Methode d'Euler:
\begin{equation}
    u_{n+1}=u_n+\Delta t * f(u_n)
\end{equation}

In [43]:
def euler_step(u, f, t) :
    """
    L'itération de la méthode d'Euler prenant u_n et retournant u_{n+1}
    """
    return u + dt*f(u, t)

In [44]:
for n in range(N-1) :
    u[n+1] = euler_step(u[n], f, t0)  # Dans notre vecteur (de vecteur) u défini trois cellules plus haut, on place les valeurs de u_{n+1} définies grâce à la methode d'Euler.
    t0 = t0 + dt

À présent, nous avons trouvé nos valeurs.

In [45]:
i=0

for vect in u :
    print( "à t =",t[i], "s" ,", u(t) =",vect)
    i+=1

à t = 0.0 s , u(t) = [ 0.  0.]
à t = 0.1 s , u(t) = [ 0.          3.35233333]
à t = 0.2 s , u(t) = [ 0.33523333  6.76273724]
à t = 0.3 s , u(t) = [  1.01150706  10.23177892]
à t = 0.4 s , u(t) = [  2.03468495  13.75999567]
à t = 0.5 s , u(t) = [  3.41068452  17.34789158]
à t = 0.6 s , u(t) = [  5.14547367  20.99593403]
à t = 0.7 s , u(t) = [  7.24506708  24.70454998]
à t = 0.8 s , u(t) = [  9.71552207  28.47412204]
à t = 0.9 s , u(t) = [ 12.56293428  32.30498423]
à t = 1.0 s , u(t) = [ 15.7934327   36.19741761]
à t = 1.1 s , u(t) = [ 19.41317446  40.15164555]
à t = 1.2 s , u(t) = [ 23.42833902  44.16782879]
à t = 1.3 s , u(t) = [ 27.8451219   48.24606024]
à t = 1.4 s , u(t) = [ 32.66972792  52.38635945]
à t = 1.5 s , u(t) = [ 37.90836387  56.58866685]
à t = 1.6 s , u(t) = [ 43.56723055  60.85283768]
à t = 1.7 s , u(t) = [ 49.65251432  65.17863564]
à t = 1.8 s , u(t) = [ 56.17037788  69.5657262 ]
à t = 1.9 s , u(t) = [ 63.1269505  74.0136697]
à t = 2.0 s , u(t) = [ 70.52831747  78.52191

*Réponse 2:*

In [46]:
v_max = 0 # Vitesse maximale de la fusée 
h_v_max = 0 # Hauteur pour laquelle la vitesse de la fusée est maximale
i = 0
for vect in u :
    while vect[1] > v_max :
        v_max = vect[1]
        h_v_max = vect[0]
        i = i+1
temps = i*dt 

print ("La vitesse maximale de la fusée est de", v_max , "m/s. Elle atteint cette vitesse en", temps, "secondes et son altitude à cet instant est de", h_v_max , "m.")

La vitesse maximale de la fusée est de 232.106133413 m/s. Elle atteint cette vitesse en 5.0 secondes et son altitude à cet instant est de 523.522834292 m.


*Réponse 3:*

In [47]:
h_max = 0 # Hauteur maximal de la fusée lors de son vol
i = 0
for vect in u :
    while vect[0] > h_max :
        h_max = vect[0]
        i = i+1
temps = i*dt
print ("La hauteur maximale de la fusée lors de son vol est de", h_max, "m. Elle atteint cette hauteur en", temps, "secondes.")

La hauteur maximale de la fusée lors de son vol est de 1334.18294543 m. Elle atteint cette hauteur en 15.600000000000001 secondes.


*Réponse 4:*

Pour répondre à cette question nous avons utilisé le fait que quand la hauteur (u[0]) est positive, la fusée n'a pas encore percuté le sol et quand elle est négative la fusée est "en dessous du sol" et donc, physiquement, l'impact a eu lieu.

In [48]:
v_imp = numpy.empty((30, 1))  #on cré un tableau (à proprement parlé "array"=vecteur) pour rangé tout le u[1] après l'impact au sol
i = 0
j = 1
for vect in u :
    if vect[0] == abs(vect[0]) :  
        j = j+1
    else :
        v_imp[i] = vect[1]
        i = i+1
temps = j*dt    
print("Après", temps, "secondes de vol, l'impact de la fusée avec le sol a lieu. A cet instant la fusée possède une vitesse de", v_imp[0], "m/s.")

Après 37.2 secondes de vol, l'impact de la fusée avec le sol a lieu. A cet instant la fusée possède une vitesse de [-86.00683498] m/s.
